# EDA анализ данных сенсоров

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler
import joblib
import warnings
warnings.filterwarnings('ignore')

# Загрузка данных

In [ ]:
df = pd.read_parquet('../kafka-lab1/data/processed/combined.parquet')
print(f"Размер данных: {df.shape}")
print(f"Колонки: {df.columns.tolist()}")

# Базовая статистика

In [ ]:
df.describe()

# Проверка пропусков

In [ ]:
df.isnull().sum()

# Распределение целевой переменной

In [ ]:
df['is_anomaly'].value_counts(normalize=True)

# Временной ряд температуры для одного сенсора

In [ ]:
sensor_data = df[df['sensor_id'] == 'SENSOR_0001'].sort_values('timestamp')
plt.figure(figsize=(15, 5))
plt.plot(sensor_data['timestamp'], sensor_data['temperature'])
plt.title('Температура сенсора SENSOR_0001 во времени')
plt.xlabel('Время')
plt.ylabel('Температура')
plt.show()

# Корреляционная матрица

In [ ]:
numeric_cols = ['temperature', 'humidity', 'pressure', 'vibration', 
                'air_quality_co2', 'battery_level', 'signal_strength']
corr_matrix = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Корреляционная матрица признаков')
plt.show()

# Распределение аномалий по часам

In [ ]:
hourly_anomalies = df.groupby('hour')['is_anomaly'].mean()
plt.figure(figsize=(12, 6))
hourly_anomalies.plot(kind='bar')
plt.title('Доля аномалий по часам')
plt.xlabel('Час')
plt.ylabel('Доля аномалий')
plt.show()

# Подготовка данных для ML

In [ ]:
feature_cols = ['temperature', 'humidity', 'pressure', 'vibration', 
                'air_quality_co2', 'battery_level', 'signal_strength',
                'hour', 'day_of_week', 'is_weekend']

X = df[feature_cols]
y = df['is_anomaly']

# Разделение на train/test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Масштабирование

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Модель 1: Random Forest

In [ ]:
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train_scaled, y_train)

# Предсказания

In [ ]:
rf_pred = rf_model.predict(X_test_scaled)
rf_proba = rf_model.predict_proba(X_test_scaled)[:, 1]

# Оценка

In [ ]:
print("Random Forest Results:")
print(classification_report(y_test, rf_pred))
print(f"ROC-AUC: {roc_auc_score(y_test, rf_proba):.4f}")

# Модель 2: Gradient Boosting

In [ ]:
gb_model = GradientBoostingClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42
)
gb_model.fit(X_train_scaled, y_train)

gb_pred = gb_model.predict(X_test_scaled)
gb_proba = gb_model.predict_proba(X_test_scaled)[:, 1]

print("\nGradient Boosting Results:")
print(classification_report(y_test, gb_pred))
print(f"ROC-AUC: {roc_auc_score(y_test, gb_proba):.4f}")

# Важность признаков

In [ ]:
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance, x='importance', y='feature')
plt.title('Важность признаков (Random Forest)')
plt.tight_layout()
plt.show()

# Сохранение лучшей модели

In [ ]:
joblib.dump(rf_model, '../kafka-lab1/models/anomaly_detector.pkl')
joblib.dump(scaler, '../kafka-lab1/models/scaler.pkl')

print("Модели сохранены в директорию ../models/")